In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from kmodes.kmodes import KModes

from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

In [11]:
df=pd.read_csv('target_enhance.csv', low_memory=False)

In [12]:
mode_val=df['source'].mode()[0]
df['source']=df['source'].fillna(mode_val)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308911 entries, 0 to 308910
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   unsubs_from      308911 non-null  object
 1   unsubs_date      308911 non-null  object
 2   revenue          308911 non-null  int64 
 3   subs_date        308911 non-null  object
 4   source           308911 non-null  object
 5   country          308911 non-null  object
 6   operator         308911 non-null  object
 7   profile_status   308911 non-null  object
 8   status           308911 non-null  int64 
 9   service_price    308911 non-null  int64 
 10  publisher        308911 non-null  object
 11  adnet            308911 non-null  object
 12  service          308911 non-null  object
 13  success_billing  308911 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 33.0+ MB


In [17]:
def kmodes_clustering(df, n_clusters=2):
    # Selecting only categorical variables
    categorical_features = df.select_dtypes(include=['object']).columns
    
    X_categorical = df[categorical_features]

    # K-Modes Implementation
    kmodes = KModes(n_clusters=n_clusters, init='Cao', verbose=1)
    df['cluster'] = kmodes.fit_predict(X_categorical)


    result_df = df[['cluster'] + list(categorical_features)]

    return result_df


result_df = kmodes_clustering(df)

result_df

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 706249.0


,cluster,unsubs_from,unsubs_date,subs_date,source,country,operator,profile_status,publisher,adnet,service
0,0,no_date,no_date,2022-10-20 19:28:02.000,sms,th,ais,active,missing,missing,gemezz
1,0,no_date,no_date,2020-08-23 16:11:02.000,sms,th,ais,active,missing,missing,gemezz
2,0,no_date,no_date,2023-08-25 14:02:05.000,ivr,th,ais,active,missing,missing,gemezz
3,0,no_date,no_date,2023-09-09 13:03:24.000,wap,th,ais,active,star,star,gemezz
4,0,no_date,no_date,2022-09-26 19:36:03.000,sms,th,ais,active,missing,missing,gemezz
...,...,...,...,...,...,...,...,...,...,...,...
308906,1,no_date,0001-01-01 00:00:00.000,2023-05-11 09:50:11.000,cp1,om,omantel,active,lumos,kee,goaly
308907,1,no_date,0001-01-01 00:00:00.000,2023-05-11 09:49:50.000,cp1,om,omantel,active,lumos,kee,goaly
308908,1,no_date,0001-01-01 00:00:00.000,2023-05-11 06:15:24.000,cp1,om,omantel,active,lumos,kee,goaly
308909,1,no_date,0001-01-01 00:00:00.000,2023-05-11 16:44:57.000,cp1,om,omantel,active,lumos,kee,goaly


In [19]:


def evaluate_kmodes(result_df):
 
    # Extract the cluster labels and categorical features from the result_df
    labels = result_df['cluster']
    categorical_features = result_df.columns[1:]

    # Label encode categorical features
    le = LabelEncoder()
    result_df_encoded = result_df.copy()
    for feature in categorical_features:
        result_df_encoded[feature] = le.fit_transform(result_df_encoded[feature])

    # Calculate silhouette score
    silhouette_avg = silhouette_score(result_df_encoded[categorical_features], labels, metric='hamming')
    
    return silhouette_avg


silhouette_score = evaluate_kmodes(result_df)
print(f"Silhouette Score: {silhouette_score}")


Silhouette Score: 0.6365263130069132
